# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [371]:
import nest_asyncio
nest_asyncio.apply()

In [372]:
from dotenv import load_dotenv
load_dotenv()

True

In [373]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [374]:
from utils.models_define import ModelName

# storing or not
STORE = False
DELETE = False
model_name = ModelName.AZURE_OPENAI

## **LLM Models Define**

In [375]:
from llama_index.core import Settings
from utils.models_define import get_llm_and_embedding

# ollama/openai/azure_openai
llm, embedding_llm = get_llm_and_embedding(model_name=model_name)

Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [376]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes
from utils.doc_store import store_docstore, load_docstore

path = './data'

if STORE:
    docs_all = read_documents(path=path)
    nodes_all = parse_nodes(documents=docs_all, llm=llm)
    docstore = store_docstore(documents=docs_all, delete=DELETE)
else:
    docstore = load_docstore()

## **Indexing & Storing**

In [377]:
from utils.vector_store import (store_vector_index, load_vector_index)

# store
if STORE:
    index_all = store_vector_index(nodes=nodes_all,embed_model=embedding_llm, delete=DELETE)

# load
else:
    index_all = load_vector_index()

Loading the index completed.


## **Querying**

### **Retriever**

In [378]:
from llama_index.retrievers.bm25 import BM25Retriever
from utils.fusion_retriever import FusionRetriever
# from llama_index.core.retrievers import RecursiveRetriever

vector_retriever_all = index_all.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore, similarity_top_k=2, verbose=True
)

fusion_retriever = FusionRetriever(
    llm=llm,
    retrievers=[vector_retriever_all, bm25_retriever],
    similarity_top_k=2,
)

### **Query Engine**

In [379]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine_all = RetrieverQueryEngine.from_args(retriever=fusion_retriever,llm=llm)

In [404]:
a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # 1/SEOJK.04/2013
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ scooter?" # Rp.140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.

z = "What is the limit of QRIS Transaction Nominal?"

query_str = n

In [405]:
response = query_engine_all.query(query_str)

100%|██████████| 6/6 [00:02<00:00,  2.70it/s]


In [406]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** Tidak ada informasi yang menyebutkan apakah KMK Nomor 462/KMK.04/1998 masih berlaku atau tidak dalam konteks ini.

## **Build Agent**

In [407]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_tools = [
    QueryEngineTool(
        query_engine=query_engine_all,
        metadata=ToolMetadata(
            name="bi_ojk",
            description="Useful for retrieving answers from all documents.",
        )
    ),
]

In [408]:
system_prompt = """
You are a compliance chatbot specializing in Indonesian regulations, including those from OJK, Bapepam, Bank Indonesia, and others.

**Guidelines:**
1. **Language:** Always respond in the user's language.
2. **Source:** Use the vector search tool for every query to find relevant information.
3. **Accuracy:** Do not generate information not present in your training data.
4. **Scope:** Only provide answers based on the following documents:
   - Peraturan OJK
   - Surat Edaran OJK
   - PPBI
   - Klasifikasi Bapepam
   - SEBI
   - Peraturan Keputusan Menteri
   - Peraturan Bapepam
   - Undang-Undang
   - Peraturan Pemerintah

**Capabilities:**
- Summarize content from the regulations.
- Answer specific questions about articles or sections.
- Utilize previous chat history and context for consistent and accurate interactions.

**Interaction Format:**
Query: {query_str}
Vector Search Results: {query_results}
Answer: {answer}
"""
# system_prompt = """
# You are a chatbot able to help answer questions about various types of regulations in Indonesia.

# You MUST always answer WITH BAHASA INDONESIA.
# You are NOT ALLOWED to hallucinate.
# You MUST FOLLOW the answer from query engine.
# You are ONLY ALLOWED to answer the question based on documents that you have been trained on.
# You have access to a query tool to help you find relevant information in the regulations database.
# Given the context information and not prior knowledge, answer the query using the query tool provided.
# Use the previous chat history, or the context above, to interact and help the user.
# ----------------------------------------------
# Query: {query_str}
# Answer:
# """

In [409]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [410]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [411]:
from utils.custom_agent import YourOpenAIAgent, CustomAgent, agent_with_memory
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent.legacy.react.base import ReActAgent

# agent = custom_agent(
#     llm=llm,
#     query_engine_tools=query_tools,
#     verbose=True,
# )

# agent = YourOpenAIAgent(
#     llm=llm,
#     tools=query_tools,
# )

# agent = agent_with_memory(
#     llm=llm,
#     tools=query_tools,
#     memory=memory,
#     system_prompt=system_prompt
# )

# agent = CustomAgent(
#     llm=llm,
#     query_engine_tools=query_tools,
#     verbose=True,
# )

# agent = OpenAIAgent.from_tools(
#     llm=llm,
#     tools=query_tools,
#     verbose=True,
#     memory=memory,
#     system_prompt=system_prompt,
#     callback_manager=callback_manager,
# )

agent = ReActAgent.from_tools(
    llm=llm,
    memory=memory,
    tools=query_tools,
    verbose=True,
    system_prompt=system_prompt,
    callback_manager=callback_manager,
)

In [412]:
# response_openai = agent_openai.chat(query_str)
# response_react = agent_react.chat(query_str)
response = agent.chat(message=query_str)

Thought: The current language of the user is Indonesian. I need to use a tool to help me answer the question.
Action: bi_ojk
Action Input: {'input': 'Apakah KMK Nomor 462/KMK.04/1998 masih berlaku'}


100%|██████████| 6/6 [00:01<00:00,  3.54it/s]


Observation: Tidak ada informasi yang menyebutkan apakah KMK Nomor 462/KMK.04/1998 masih berlaku atau tidak dalam konteks ini.
Thought: I cannot answer the question with the provided tools.
Answer: Tidak ada informasi yang menyebutkan apakah KMK Nomor 462/KMK.04/1998 masih berlaku atau tidak dalam konteks ini.
**********
Trace: chat
    |_llm -> 1.877204 seconds
    |_function_call -> 4.593356 seconds
    |_llm -> 1.379535 seconds
**********


In [413]:
from llama_index.core.response.notebook_utils import display_response

display_response(response=response)

**`Final Response:`** Tidak ada informasi yang menyebutkan apakah KMK Nomor 462/KMK.04/1998 masih berlaku atau tidak dalam konteks ini.

In [414]:
for node in response.source_nodes:
    print("**********")
    print("id: ", node.id_)
    print("content:\n", node.node.get_content())
    if node.score:
        print("reranking score: ", node.score)
    if node.node.metadata.get("retrieval_score"):
        print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("**********")
    print("\n")

**********
id:  0615b9f7-6375-4a61-9360-a9639ec03269
content:
 Peraturan Pemerintah Nomor 45 Tahun 1995 tentangPenyelenggaraan Kegiatan di Bidang Pasar Modal (Lembaran Negara Republik Indonesia Tahun 1995 Nomor 86, Tambahan Lembaran Negara Nomor 3617); MEMUTUSKAN : Menetapkan : P E R AT U R A N  P E M E R I N TA H  T E N TA N G  P E R U B A H A N  ATA S         PERATURAN PEMERINTAH NOMOR 45 TAHUN 1995 TENTANG       PENYELENGGARAAN KEGIATAN DI BIDANG PASAR MODAL. II- 2 PP. No. : 12 Tahun 2004 Pasal I Mengubah ketentuan Pasal 8 dalam Peraturan Pemerintah Nomor 45 Tahun 1995 tentang Penyelenggaraan Kegiatan Di Bidang Pasar Modal, sehingga keseluruhan Pasal 8 berbunyi sebagai berikut : Pasal 8  (1)  Perusahaan Efek yang telah menjadi pemegang saham Bursa Efek tetapi kemudian tidak lagi memenuhi syarat untuk menjadi Anggota Bursa Efek atau tidak lagi menjadi Anggota Bursa Efek, wajib mengalihkan saham Bursa Efek yang dimilikinya kepada Perusahaan Efek lain yang memenuhi persyaratan sebagai 

In [415]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)